# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770494568963                   -0.52    9.0         
  2   -2.771694438443       -2.92       -1.32    1.0    219ms
  3   -2.771714348442       -4.70       -2.45    1.0    147ms
  4   -2.771714643993       -6.53       -3.16    1.0    152ms
  5   -2.771714714760       -7.15       -4.11    2.0    247ms
  6   -2.771714715147       -9.41       -4.59    1.0    155ms
  7   -2.771714715248      -10.00       -5.81    1.0    159ms
  8   -2.771714715250      -11.78       -6.08    2.0    206ms
  9   -2.771714715250      -12.89       -6.52    1.0    169ms
 10   -2.771714715250      -13.82       -7.74    2.0    204ms
 11   -2.771714715250      -14.65       -8.47    2.0    183ms


-0.00013457368870221434

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770518611989                   -0.52    9.0         
  2   -2.771779884979       -2.90       -1.32    1.0    205ms
  3   -2.771801683569       -4.66       -2.45    1.0    212ms
  4   -2.771802000376       -6.50       -3.15    1.0    220ms
  5   -2.771802074249       -7.13       -4.22    2.0    223ms
  6   -2.771802074318      -10.16       -4.32    1.0    160ms
  7   -2.771802074472       -9.81       -5.34    1.0    187ms
  8   -2.771802074476      -11.43       -5.45    2.0    185ms
  9   -2.771802074476      -12.54       -6.07    1.0    190ms
 10   -2.771802074476      -13.73       -6.24    1.0    169ms
 11   -2.771802074476      -13.58       -6.61    1.0    173ms
 12   -2.771802074476      -13.86       -7.31    2.0    197ms
 13   -2.771802074476      -14.05       -7.60    1.0    186ms
 14   -2.771802074476   +  -13.81       -8.45    1.0    181ms


0.0176122212091171

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457368870221434
Displaced dipole:  0.0176122212091171
Polarizability :   1.7746794897819316


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920658522e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551230056e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852766715723e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593853397e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787446174e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.273879846436e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.697574352418e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.697574352418e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.505074797936e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.225815523307e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.675840787914e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 5.863610008118e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 5.863610008118e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.